# All image csv files

In [1]:
import os
import csv

def create_csv(image_folder, csv_file_path):
    # CSV header
    fieldnames = ['HospitalID', 'PatientID', 'ImageID', 'ImagePath', 'Label']
    
    # open the CSV file in write mode
    with open(csv_file_path, 'w', newline='') as csvfile:
        # Create a CSV writer object and write the header
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for filename in sorted(os.listdir(image_folder)):
            # complete image file path
            image_path = os.path.join(image_folder, filename)

            # filename to extract HospitalID, PatientID, image number, and image label
            parts = filename.split('_')
            hospital_id = parts[3]
            patient_id = parts[1]
            image_number = parts[6]
            image_label = parts[4]

            # write data into the CSV file
            writer.writerow({
                'HospitalID': hospital_id,
                'PatientID': patient_id,
                'ImageID': image_number,
                'ImagePath': image_path,
                'Label': image_label
            })

    # created and saved
    print("CSV file has been created and saved to:", csv_file_path)


image_folder = "/local/data1/honzh073/data/8bit_down224"
csv_file_path = "/local/data1/honzh073/local_repository/FL/code/6_test_global_model/csv_files/all_image.csv"
create_csv(image_folder, csv_file_path)


CSV file has been created and saved to: /local/data1/honzh073/local_repository/FL/code/6_test_global_model/csv_files/all_image.csv


# Custom Dataset and test 100 times average

In [2]:
# Custom dataset class
import os
import csv
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms
from sklearn.metrics import confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
import torch.nn.functional as F

# custom dataset on csv files
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = []
        self.labels = []  # Store labels separately
        self.patient_ids = []  # Store patient IDs separately
        self.transform = transform
        
        # read csv
        with open(csv_file, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                image_path = row['ImagePath']
                label = row['Label']
                patient_id = row['PatientID']  # Assuming 'PatientID' is the column name in your CSV file

                if label == 'NFF':
                    label = 0
                elif label == 'AFF':
                    label = 1
                else:
                    raise ValueError("Invalid label in CSV file.")
                self.data.append((image_path, label))
                self.labels.append(label)
                self.patient_ids.append(patient_id)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from sklearn.model_selection import StratifiedKFold
import random
import pandas as pd
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

# ------------------------------------------------------------------------------------------------------
def generate_random_pick_dataset(seed, excluded_hospitals, input_csv_path, output_csv_path):
    random.seed(seed)
    
    # Read the original CSV file
    df = pd.read_csv(input_csv_path)
    
    # Randomly select patients from non-excluded hospitals
    selected_data = []
    for hospital_id in df['HospitalID'].unique():
        if hospital_id not in excluded_hospitals:
            patients = df[df['HospitalID'] == hospital_id]['PatientID'].unique()
            selected_patients = random.sample(list(patients), min(3, len(patients)))
            for patient_id in selected_patients:
                patient_data = df[(df['HospitalID'] == hospital_id) & (df['PatientID'] == patient_id)]
                selected_data.append(patient_data)
    
    # Concatenate selected data to create a new DataFrame
    selected_df = pd.concat(selected_data)
    
    # Save the new DataFrame as a CSV file
    selected_df.to_csv(output_csv_path, index=False)
    
    # Return the output file path
    return output_csv_path
# ------------------------------------------------------------------------------------------------------
def test_model(model, test_dataset, batch_size):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
    model = model.to(device)

    model.eval()
    
    correct_test_nff = 0
    correct_test_aff = 0
    total_nff = 0
    total_aff = 0
    total_test = 0
    test_loss = 0
    
    criterion = nn.CrossEntropyLoss()
    
    with torch.no_grad():
        all_predictions = []
        all_labels = []
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_test += labels.size(0)
            total_nff += (labels == 0).sum().item()
            total_aff += (labels == 1).sum().item()
            correct_test_nff += ((predicted == labels) & (labels == 0)).sum().item()
            correct_test_aff += ((predicted == labels) & (labels == 1)).sum().item()
            test_loss += loss.item()

    test_accuracy_nff = 100 * correct_test_nff / total_nff
    test_accuracy_aff = 100 * correct_test_aff / total_aff
    test_accuracy_total = 100 * (correct_test_nff + correct_test_aff) / total_test
    test_loss /= len(test_loader)
    
    auc_score = roc_auc_score(all_labels, all_predictions)
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    class_labels = {0: 'NFF', 1: 'AFF'}
    classification_rep = classification_report(all_labels, all_predictions, target_names=[class_labels[i] for i in range(len(class_labels))])
    
    # Store metrics in a dictionary
    metrics = {
        "Test Loss": test_loss,
        "Total Test Accuracy": test_accuracy_total,
        "Test Accuracy (NFF)": test_accuracy_nff,
        "Test Accuracy (AFF)": test_accuracy_aff,
        "AUC": auc_score,
        "Confusion Matrix": conf_matrix,
        "Classification Report": classification_rep
    }
    return metrics
# ------------------------------------------------------------------------------------------------------
# Set random seed for reproducibility
def run_100_times(model):
    # Define the excluded hospitals and file paths
    excluded_hospitals = [18, 43, 55, 100]
    input_csv_path = '/local/data1/honzh073/local_repository/FL/code/6_test_global_model/csv_files/all_image.csv'
    output_csv_path = '/local/data1/honzh073/local_repository/FL/code/6_test_global_model/csv_files/randompick_patient_data.csv'

    # Initialize lists to store metrics
    test_metrics_list = []

    # Generate random seeds from 1 to 100
    for seed in range(1, 101):
        print(f"Seed: {seed} / 100", end='\r')
        # Generate random pick dataset
        randompick_patient_data_path = generate_random_pick_dataset(seed, excluded_hospitals, input_csv_path, output_csv_path)
        
        # Load test_dataset using random pick data
        test_dataset = CustomDataset(randompick_patient_data_path, transform=test_transform)  # Define your CustomDataset and transform
        
        # Perform testing and collect metrics
        test_metrics = test_model(model, test_dataset, batch_size=8)  # Use your batch size
        test_metrics_list.append(test_metrics)

    # Calculate mean and standard deviation of metrics
    test_loss_values = [metrics['Test Loss'] for metrics in test_metrics_list]
    total_accuracy_values = [metrics['Total Test Accuracy'] for metrics in test_metrics_list]
    nff_accuracy_values = [metrics['Test Accuracy (NFF)'] for metrics in test_metrics_list]
    aff_accuracy_values = [metrics['Test Accuracy (AFF)'] for metrics in test_metrics_list]
    auc_values = [metrics['AUC'] for metrics in test_metrics_list]

    mean_test_loss = sum(test_loss_values) / len(test_loss_values)
    mean_total_accuracy = sum(total_accuracy_values) / len(total_accuracy_values)
    mean_nff_accuracy = sum(nff_accuracy_values) / len(nff_accuracy_values)
    mean_aff_accuracy = sum(aff_accuracy_values) / len(aff_accuracy_values)
    mean_auc = sum(auc_values) / len(auc_values)

    std_test_loss = (sum((x - mean_test_loss) ** 2 for x in test_loss_values) / len(test_loss_values)) ** 0.5
    std_total_accuracy = (sum((x - mean_total_accuracy) ** 2 for x in total_accuracy_values) / len(total_accuracy_values)) ** 0.5
    std_nff_accuracy = (sum((x - mean_nff_accuracy) ** 2 for x in nff_accuracy_values) / len(nff_accuracy_values)) ** 0.5
    std_aff_accuracy = (sum((x - mean_aff_accuracy) ** 2 for x in aff_accuracy_values) / len(aff_accuracy_values)) ** 0.5
    std_auc = (sum((x - mean_auc) ** 2 for x in auc_values) / len(auc_values)) ** 0.5

    # Print mean and standard deviation values
    # Print mean and standard deviation values with 4 decimal places
    print("Mean Test Loss:", round(mean_test_loss, 4))
    print("Standard Deviation Test Loss:", round(std_test_loss, 4))
    print("Mean Total Accuracy:", round(mean_total_accuracy, 4))
    print("Standard Deviation Total Accuracy:", round(std_total_accuracy, 4))
    print("Mean NFF Accuracy:", round(mean_nff_accuracy, 4))
    print("Standard Deviation NFF Accuracy:", round(std_nff_accuracy, 4))
    print("Mean AFF Accuracy:", round(mean_aff_accuracy, 4))
    print("Standard Deviation AFF Accuracy:", round(std_aff_accuracy, 4))
    print("Mean AUC:", round(mean_auc, 4))
    print("Standard Deviation AUC:", round(std_auc, 4))



# VGG 19

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import vgg19, VGG19_Weights

torch.hub.set_dir('/local/data1/honzh073/download/TORCH_PRETRAINED')

# models from job, 50% freezed
class CustomNet(nn.Module):
    def __init__(self, num_classes=2, freeze_percentage=0.5):
        super(CustomNet, self).__init__()

        # Load the pre-trained VGG19 model
        vgg19_model = models.vgg19(weights=VGG19_Weights.DEFAULT)

        # Extract features (all layers except the classifier)
        self.features = nn.Sequential(*list(vgg19_model.features.children()))

        # Calculate the index where to split the layers
        total_layers = len(list(self.features.children()))
        split_idx = int(total_layers * freeze_percentage)

        # Split the layers into groups for freezing and non-freezing
        children = list(self.features.children())
        self.frozen_features = nn.Sequential(*children[:split_idx])
        self.unfrozen_features = nn.Sequential(*children[split_idx:])

        # Freeze layers in self.frozen_features
        for param in self.frozen_features.parameters():
            param.requires_grad = False

        # Define the final fully connected layer
        in_features = vgg19_model.classifier[0].in_features
        self.classifier = nn.Linear(in_features, num_classes)  # VGG19's last feature map has 512 channels

    def forward(self, x):
        x = self.frozen_features(x)
        x = self.unfrozen_features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# create net
model = CustomNet(num_classes=2)  # num_classes 2

# checkpoint
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# vgg19_freeze_Adam_200R_1E
global_path = '/local/data1/honzh073/data/suzuki.ad.liu.se/transfer/vgg19_freeze_Adam_200R_1E/workspace/app_server/FL_global_model.pt'

checkpoint = torch.load(global_path, map_location=device)

# print(checkpoint.keys())

# modify checkpoint keys，match model
# load model weights
state_dict = checkpoint['model']
modified_state_dict = {}
for key, value in state_dict.items():
    # modify key names，match model layers
    new_key = key.replace("model.", "")
    modified_state_dict[new_key] = value

# load weights to model
model.load_state_dict(modified_state_dict)
model = model.to(device)
model.eval()

run_100_times(model)


Mean Test Loss: 0.5736
Standard Deviation Test Loss: 0.0977
Mean Total Accuracy: 88.1745
Standard Deviation Total Accuracy: 1.4956
Mean NFF Accuracy: 91.6996
Standard Deviation NFF Accuracy: 1.3189
Mean AFF Accuracy: 70.8532
Standard Deviation AFF Accuracy: 4.7077
Mean AUC: 0.8128
Standard Deviation AUC: 0.0245


# ResNet 50


In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import resnet101, ResNet101_Weights
from torchvision.models import resnet50, ResNet50_Weights

torch.hub.set_dir('/local/data1/honzh073/download/TORCH_PRETRAINED')

class CustomNet(nn.Module):
    def __init__(self, num_classes=2, freeze_percentage=0.5):
        super(CustomNet, self).__init__()

        self.features = models.resnet50(weights=ResNet50_Weights.DEFAULT)

        # Calculate the index where to split the layerszh
        total_layers = len(list(self.features.children()))
        split_idx = int(total_layers * freeze_percentage)

        # Split the layers into groups for freezing and non-freezing
        children = list(self.features.children())
        self.frozen_features = nn.Sequential(*children[:split_idx])
        self.unfrozen_features = nn.Sequential(*children[split_idx:-1])  # Exclude the last layer

        # Freeze layers in self.frozen_features
        for param in self.frozen_features.parameters():
            param.requires_grad = False

        # Define the final fully connected layer
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.frozen_features(x)
        x = self.unfrozen_features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
# create net
model = CustomNet(num_classes=2)  # num_classes 2

# checkpoint
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# resnet50_freeze_Adam_200R_1E
global_path = '/local/data1/honzh073/data/suzuki.ad.liu.se/transfer/resnet50_freeze_Adam_200R_1E/workspace/app_server/FL_global_model.pt'

checkpoint = torch.load(global_path, map_location=device)

# print(checkpoint.keys())

# modify checkpoint keys，match model
# load model weights
state_dict = checkpoint['model']
modified_state_dict = {}
for key, value in state_dict.items():
    # modify key names，match model layers
    new_key = key.replace("model.", "")
    modified_state_dict[new_key] = value

# load weights to model
model.load_state_dict(modified_state_dict)
model = model.to(device)
model.eval()

run_100_times(model)


Mean Test Loss: 0.3845
Standard Deviation Test Loss: 0.0416
Mean Total Accuracy: 87.4984
Standard Deviation Total Accuracy: 1.4084
Mean NFF Accuracy: 91.8517
Standard Deviation NFF Accuracy: 1.0737
Mean AFF Accuracy: 66.1412
Standard Deviation AFF Accuracy: 5.0764
Mean AUC: 0.79
Standard Deviation AUC: 0.0263


# ResNet 101

In [6]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import resnet101, ResNet101_Weights
from torchvision.models import resnet50, ResNet50_Weights

torch.hub.set_dir('/local/data1/honzh073/download/TORCH_PRETRAINED')

class CustomNet(nn.Module):
    def __init__(self, num_classes=2, freeze_percentage=0.5):
        super(CustomNet, self).__init__()

        self.features = models.resnet101(weights=ResNet101_Weights.DEFAULT)

        # Calculate the index where to split the layers
        total_layers = len(list(self.features.children()))
        split_idx = int(total_layers * freeze_percentage)

        # Split the layers into groups for freezing and non-freezing
        children = list(self.features.children())
        self.frozen_features = nn.Sequential(*children[:split_idx])
        self.unfrozen_features = nn.Sequential(*children[split_idx:-1])  # Exclude the last layer

        # Freeze layers in self.frozen_features
        for param in self.frozen_features.parameters():
            param.requires_grad = False

        # Define the final fully connected layer
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.frozen_features(x)
        x = self.unfrozen_features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
# create net
model = CustomNet(num_classes=2)  # num_classes 2

# checkpoint
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# resnet101_freeze_Adam_200R_1E
global_path = '/local/data1/honzh073/data/suzuki.ad.liu.se/transfer/resnet101_freeze_Adam_200R_1E/workspace/app_server/FL_global_model.pt'

checkpoint = torch.load(global_path, map_location=device)

# print(checkpoint.keys())

# modify checkpoint keys，match model
# load model weights
state_dict = checkpoint['model']
modified_state_dict = {}
for key, value in state_dict.items():
    # modify key names，match model layers
    new_key = key.replace("model.", "")
    modified_state_dict[new_key] = value

# load weights to model
model.load_state_dict(modified_state_dict)
model = model.to(device)
model.eval()

run_100_times(model)


Mean Test Loss: 0.5193
Standard Deviation Test Loss: 0.0929
Mean Total Accuracy: 89.291
Standard Deviation Total Accuracy: 1.3926
Mean NFF Accuracy: 94.1199
Standard Deviation NFF Accuracy: 1.0247
Mean AFF Accuracy: 65.5622
Standard Deviation AFF Accuracy: 4.7336
Mean AUC: 0.7984
Standard Deviation AUC: 0.0239


# ResNet152

In [10]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import resnet152, ResNet152_Weights

torch.hub.set_dir('/local/data1/honzh073/download/TORCH_PRETRAINED')

class CustomNet(nn.Module):
    def __init__(self, num_classes=2, freeze_percentage=0.5):
        super(CustomNet, self).__init__()

        self.features = models.resnet152(weights=ResNet152_Weights.DEFAULT)

        # Calculate the index where to split the layers
        total_layers = len(list(self.features.children()))
        split_idx = int(total_layers * freeze_percentage)

        # Split the layers into groups for freezing and non-freezing
        children = list(self.features.children())
        self.frozen_features = nn.Sequential(*children[:split_idx])
        self.unfrozen_features = nn.Sequential(*children[split_idx:-1])  # Exclude the last layer

        # Freeze layers in self.frozen_features
        for param in self.frozen_features.parameters():
            param.requires_grad = False

        # Define the final fully connected layer
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.frozen_features(x)
        x = self.unfrozen_features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
# create net
model = CustomNet(num_classes=2)  # num_classes 2

# checkpoint
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# resnet152_freeze_Adam_200R_1E
global_path = '/local/data1/honzh073/data/suzuki.ad.liu.se/34e00b5d-518e-42d5-8fe3-0f0ee5b44959/app_server/FL_global_model.pt'

checkpoint = torch.load(global_path, map_location=device)

# print(checkpoint.keys())

# modify checkpoint keys，match model
# load model weights
state_dict = checkpoint['model']
modified_state_dict = {}
for key, value in state_dict.items():
    # modify key names，match model layers
    new_key = key.replace("model.", "")
    modified_state_dict[new_key] = value

# load weights to model
model.load_state_dict(modified_state_dict)
model = model.to(device)
model.eval()

run_100_times(model)


Mean Test Loss: 0.3875
Standard Deviation Test Loss: 0.0741
Mean Total Accuracy: 91.1242
Standard Deviation Total Accuracy: 1.3936
Mean NFF Accuracy: 95.3207
Standard Deviation NFF Accuracy: 0.8778
Mean AFF Accuracy: 70.5477
Standard Deviation AFF Accuracy: 5.448
Mean AUC: 0.8293
Standard Deviation AUC: 0.0281


# DenseNet 161


In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision.models import densenet161, DenseNet161_Weights

torch.hub.set_dir('/local/data1/honzh073/download/TORCH_PRETRAINED')

class CustomNet(nn.Module):
    def __init__(self, num_classes=2, freeze_percentage=0.5):
        super(CustomNet, self).__init__()

        self.features = models.densenet161(weights=DenseNet161_Weights.DEFAULT)

        # Calculate the index where to split the layers
        total_layers = len(list(self.features.children()))
        split_idx = int(total_layers * freeze_percentage)

        # Split the layers into groups for freezing and non-freezing
        children = list(self.features.children())
        self.frozen_features = nn.Sequential(*children[:split_idx])
        self.unfrozen_features = nn.Sequential(*children[split_idx:-1])  # Exclude the last layer

        # Freeze layers in self.frozen_features
        for param in self.frozen_features.parameters():
            param.requires_grad = False

        # Define the final fully connected layer
        self.fc = nn.Linear(2208, num_classes)  # DenseNet-161 has 2208 output features

    def forward(self, x):
        x = self.frozen_features(x)
        x = self.unfrozen_features(x)
        x = nn.functional.adaptive_avg_pool2d(x, (1, 1))  # Global average pooling
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


# create net
model = CustomNet(num_classes=2)  # num_classes 2

# checkpoint
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# resnet101_freeze_Adam_200R_1E
global_path = '/local/data1/honzh073/data/suzuki.ad.liu.se/transfer/densenet161_freeze_Adam_200R1E/workspace/app_server/FL_global_model.pt'

checkpoint = torch.load(global_path, map_location=device)

# print(checkpoint.keys())

# modify checkpoint keys，match model
# load model weights
state_dict = checkpoint['model']
modified_state_dict = {}
for key, value in state_dict.items():
    # modify key names，match model layers
    new_key = key.replace("model.", "")
    modified_state_dict[new_key] = value

# load weights to model
model.load_state_dict(modified_state_dict)
model = model.to(device)
model.eval()

run_100_times(model)


Mean Test Loss: 0.4327
Standard Deviation Test Loss: 0.0272
Mean Total Accuracy: 83.8643
Standard Deviation Total Accuracy: 2.198
Mean NFF Accuracy: 98.1343
Standard Deviation NFF Accuracy: 0.415
Mean AFF Accuracy: 13.9229
Standard Deviation AFF Accuracy: 2.6984
Mean AUC: 0.5603
Standard Deviation AUC: 0.0139
